# Step 1 : Imports

In [ ]:
import torch
import torch.nn as nn # Neural Network modules, loss functions, Activation functions
import torch.optim as optim # Optimizers like SGD, ADAM, etc
import torch.nn.functional as F # Activation functions like tanh, relu, etc
from torch.utils.data import DataLoader # for mini batches
import torchvision.transforms as transforms # for transformations on dataset

import pandas as pd
import numpy as np

# Step 2 : Create a CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) # same conv
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) # same conv
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.fc1 = nn.Linear(32*3*3, 64)
        self.dropout = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Step 3 : Set device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.get_device_name(0)

# Step 4 : Hyperparameters

In [ ]:
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 10

# Step 5 : Load Data

In [ ]:
train_data = pd.read_csv("../input/digit-recognizer/train.csv")
train_data.head()

In [ ]:
X_train = train_data.iloc[:, 1:]
y_train = train_data.iloc[:, 0]

X_train.head()

In [ ]:
y_train.head()

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_train = X_train.values.reshape(42000, 1, 28, 28)
X_train.shape

In [ ]:
class ClassificationDataset:
    def __init__(self, X, y):
        
        self.X = X
        self.y = y
        print(X.shape)
        print(y.shape)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, item):
        
        return {
            "image" : torch.tensor(self.X[item], dtype=torch.float),
            "targets": torch.tensor(self.y[item], dtype=torch.long)
        }

In [ ]:
train_dataset = ClassificationDataset(X_train, y_train)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=64, shuffle=True
)

# Step 6 : Init Network

In [ ]:
model = CNN().to(device)

# Step 7 : Loss and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Step 8 : Train Network

In [ ]:
for data in train_loader:
    
    print(data["image"].shape)
    print(data["targets"].shape)
    break


In [ ]:
for epoch in range(num_epoch):
    for batch in train_loader:
        # Get data to cuda if possible
        data = batch["image"].to(device=device)
        targets = batch["targets"].to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad() # making grad to 0
        # so that it doesn't use the gradient from the previous batch
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

## Step 9 : Check Accuarcy

In [ ]:
def check_accuracy(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for batch in train_loader:
            # Get data to cuda if possible
            x = batch["image"].to(device=device)
            y = batch["targets"].to(device=device)
            
            scores = model(x) # 64 X 10
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples) * 100:.2f}")

    model.train()

In [ ]:
check_accuracy(train_loader, model)

## Step 10 : Testing and Submission

In [ ]:
test = pd.read_csv("../input/digit-recognizer/test.csv")
test.head()

In [ ]:
X_test = test.values
X_test.shape

In [ ]:
X_test = X_test.reshape(28000, 1, 28, 28)
print(X_test.shape)

In [ ]:
class ClassificationTestDataset:
    def __init__(self, X):
        
        self.X = X
        print(X.shape)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, item):
        
        return {
            "image" : torch.tensor(self.X[item], dtype=torch.float)
        }

In [ ]:
test_dataset = ClassificationTestDataset(X_test)

test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=64, shuffle=False
)

In [ ]:
preds = []

model.eval()

for data in test_loader:
    x = data["image"].to(device=device)
    scores = model(x)
    _, predictions = scores.max(1)
    for p in predictions:
        preds.append(p.item())
    
model.train()

In [ ]:
print(preds[:10])

In [ ]:
sub = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
sub.head()

In [ ]:
len(sub)

In [ ]:
sub["Label"] = preds

In [ ]:
sub.to_csv("submission.csv", index=False)